# **🐬🧶💙🎐👘Evaluation using fuzzy logic

In [ ]:
import pandas as pd

# นำเข้าไฟล์ generated_texts.csv
file_path = '/content/fuzzy_test5 - Sheet1.csv'  # เปลี่ยนเป็นพาธที่ถูกต้อง
df = pd.read_csv(file_path)

# ดูข้อมูลในไฟล์
df.head()


,text,label,generate-sample-1,generate-sample-2,generate-sample-3,generate-sample-4,generate-sample-5
0,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...
1,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> งวด <OPINION> ดัน <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...
2,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> พลิก <OPINION> กำไร <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...
3,NOBLE เล็งเปิดตัว 2 โครงการท้ายปี มูลค่า 3.10 ...,<SYMBOL> NOBLE <ASPECT> กำไร <OPINION> ทะลัก <...,<SYMBOL> NOBLE <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBEL <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไรทะลั...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> เล็งเปิดต...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไร <PO...
4,สรุปกระดานรายใหญ่เช้านี้ BGRIM มูลค่าสูงสุด 9 ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สูง...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> มูล...


In [ ]:
!pip install fuzzywuzzy


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
import unicodedata

# ฟังก์ชันทำความสะอาดข้อความ
def clean_text(text):
    text = unicodedata.normalize('NFC', text)
    text = text.replace("('", '').replace("</s>", '').replace("',)", '').strip()
    return text

# ฟังก์ชันตรวจสอบด้วย Fuzzy Matching
def fuzzy_compare_texts(label, generated, threshold=90):
    cleaned_label = clean_text(label)
    cleaned_generated = clean_text(generated)
    similarity_score = fuzz.ratio(cleaned_label, cleaned_generated)
    return similarity_score >= threshold

# ฟังก์ชันที่จะตรวจสอบ generated-sample สำหรับข่าวทั้งหมด
def check_samples_for_news_group(news_group, sample_columns, threshold=90):
    results = {}

    for sample in sample_columns:
        # ตรวจสอบว่าผลลัพธ์จาก generated-sample ใดตรงกับ label ใดหรือไม่
        any_match = False
        for _, row in news_group.iterrows():
            label = clean_text(row['label'])
            generated_sample = clean_text(row[sample])
            if fuzzy_compare_texts(label, generated_sample, threshold):
                any_match = True
                break

        # ถ้าเจอ match ใด ๆ ให้แถวทั้งหมดในกลุ่มนี้เป็น True
        results[sample] = any_match

    # เพิ่มคอลัมน์ใหม่สำหรับผลลัพธ์ของแต่ละ generated-sample โดยการทำสำเนาข้อมูลเดิม
    for sample in sample_columns:
        news_group[f'{sample}_correct'] = results[sample]

    return news_group

# รายชื่อคอลัมน์ generate-sample ที่ต้องการตรวจสอบ
sample_columns = ['generate-sample-1', 'generate-sample-2', 'generate-sample-3', 'generate-sample-4', 'generate-sample-5']

# นำข่าวกลุ่มเดียวกันมาตรวจสอบ แต่รักษาโครงสร้างของ DataFrame ไว้
df = df.groupby('text', group_keys=False).apply(check_samples_for_news_group, sample_columns=sample_columns)

# แสดงผลลัพธ์
df.head()


<ipython-input-29-0a5311196617>:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('text', group_keys=False).apply(check_samples_for_news_group, sample_columns=sample_columns)


,text,label,generate-sample-1,generate-sample-2,generate-sample-3,generate-sample-4,generate-sample-5,generate-sample-1_correct,generate-sample-2_correct,generate-sample-3_correct,generate-sample-4_correct,generate-sample-5_correct
0,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
1,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> งวด <OPINION> ดัน <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
2,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> พลิก <OPINION> กำไร <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
3,NOBLE เล็งเปิดตัว 2 โครงการท้ายปี มูลค่า 3.10 ...,<SYMBOL> NOBLE <ASPECT> กำไร <OPINION> ทะลัก <...,<SYMBOL> NOBLE <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBEL <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไรทะลั...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> เล็งเปิดต...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไร <PO...,True,True,False,False,False
4,สรุปกระดานรายใหญ่เช้านี้ BGRIM มูลค่าสูงสุด 9 ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สูง...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> มูล...,True,False,False,True,False


In [ ]:
# ฟังก์ชันคำนวณเปอร์เซ็นต์ความถูกต้องในแต่ละคอลัมน์
def calculate_accuracy(df, sample_columns):
    accuracy = {}
    total_rows = len(df)

    for sample in sample_columns:
        correct_count = df[f'{sample}_correct'].sum()
        accuracy[sample] = (correct_count / total_rows) * 100

    return accuracy

# รายชื่อคอลัมน์ generate-sample ที่ต้องการตรวจสอบ
sample_columns = ['generate-sample-1', 'generate-sample-2', 'generate-sample-3', 'generate-sample-4', 'generate-sample-5']

# นำข่าวกลุ่มเดียวกันมาตรวจสอบ แต่รักษาโครงสร้างของ DataFrame ไว้
df = df.groupby('text', group_keys=False).apply(check_samples_for_news_group, sample_columns=sample_columns)

# คำนวณเปอร์เซ็นต์ความถูกต้องของแต่ละ generated-sample
accuracy = calculate_accuracy(df, sample_columns)

# แสดงผลลัพธ์เปอร์เซ็นต์
print("Accuracy percentage for each generated sample:")
for sample, acc in accuracy.items():
    print(f"{sample}: {acc:.2f}%")

# แสดงผลลัพธ์ DataFrame
df.head()

Accuracy percentage for each generated sample:
generate-sample-1: 86.23%
generate-sample-2: 79.21%
generate-sample-3: 76.03%
generate-sample-4: 71.13%
generate-sample-5: 70.20%


<ipython-input-30-5d822a415435>:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('text', group_keys=False).apply(check_samples_for_news_group, sample_columns=sample_columns)


,text,label,generate-sample-1,generate-sample-2,generate-sample-3,generate-sample-4,generate-sample-5,generate-sample-1_correct,generate-sample-2_correct,generate-sample-3_correct,generate-sample-4_correct,generate-sample-5_correct
0,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
1,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> งวด <OPINION> ดัน <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
2,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> พลิก <OPINION> กำไร <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
3,NOBLE เล็งเปิดตัว 2 โครงการท้ายปี มูลค่า 3.10 ...,<SYMBOL> NOBLE <ASPECT> กำไร <OPINION> ทะลัก <...,<SYMBOL> NOBLE <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBEL <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไรทะลั...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> เล็งเปิดต...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไร <PO...,True,True,False,False,False
4,สรุปกระดานรายใหญ่เช้านี้ BGRIM มูลค่าสูงสุด 9 ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สูง...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> มูล...,True,False,False,True,False


In [ ]:
# ฟังก์ชันคำนวณเปอร์เซ็นต์ความถูกต้องในแต่ละคอลัมน์
def calculate_accuracy(df, sample_columns):
    accuracy = {}
    total_rows = len(df)

    for sample in sample_columns:
        correct_count = df[f'{sample}_correct'].sum()
        accuracy[sample] = (correct_count / total_rows) * 100

    # คำนวณเปอร์เซ็นต์เฉลี่ยของทุก generated-sample
    average_accuracy = sum(accuracy.values()) / len(sample_columns)

    return accuracy, average_accuracy

# รายชื่อคอลัมน์ generate-sample ที่ต้องการตรวจสอบ
sample_columns = ['generate-sample-1', 'generate-sample-2', 'generate-sample-3', 'generate-sample-4', 'generate-sample-5']

# นำข่าวกลุ่มเดียวกันมาตรวจสอบ แต่รักษาโครงสร้างของ DataFrame ไว้
df = df.groupby('text', group_keys=False).apply(check_samples_for_news_group, sample_columns=sample_columns)

# คำนวณเปอร์เซ็นต์ความถูกต้องของแต่ละ generated-sample และค่าเฉลี่ย
accuracy, average_accuracy = calculate_accuracy(df, sample_columns)

# แสดงผลลัพธ์เปอร์เซ็นต์
print("Accuracy percentage for each generated sample:")
for sample, acc in accuracy.items():
    print(f"{sample}: {acc:.2f}%")

# แสดงเปอร์เซ็นต์เฉลี่ยของทั้งหมด
print(f"\nAverage accuracy across all generated samples: {average_accuracy:.2f}%")

# แสดงผลลัพธ์ DataFrame
df.head()


Accuracy percentage for each generated sample:
generate-sample-1: 86.23%
generate-sample-2: 79.21%
generate-sample-3: 76.03%
generate-sample-4: 71.13%
generate-sample-5: 70.20%

Average accuracy across all generated samples: 76.56%


<ipython-input-33-2f58d0fedd99>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('text', group_keys=False).apply(check_samples_for_news_group, sample_columns=sample_columns)


,text,label,generate-sample-1,generate-sample-2,generate-sample-3,generate-sample-4,generate-sample-5,generate-sample-1_correct,generate-sample-2_correct,generate-sample-3_correct,generate-sample-4_correct,generate-sample-5_correct
0,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
1,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> งวด <OPINION> ดัน <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
2,BAFS รายได้การบินพุ่ง ดันงวด 9 เดือนพลิกกำไร 6...,<SYMBOL> BAFS <ASPECT> พลิก <OPINION> กำไร <POS>,<SYMBOL> BAFS <ASPECT> รายได้ <OPINION> พุ่ง <...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> ดัน <PO...,<SYMBOL> BAFS <ASPECT> รายได้การบิน <OPINION> ...,<SYMBOL> BAFS <ASPECT> กําไร <OPINION> พลิก <P...,<SYMBOL> BAFS <ASPECT> ผลประกอบการ <OPINION> พ...,True,True,True,False,False
3,NOBLE เล็งเปิดตัว 2 โครงการท้ายปี มูลค่า 3.10 ...,<SYMBOL> NOBLE <ASPECT> กำไร <OPINION> ทะลัก <...,<SYMBOL> NOBLE <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBEL <ASPECT> กําไร <OPINION> ทะลัก ...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไรทะลั...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> เล็งเปิดต...,<SYMBOL> NOBLE <ASPECT> งบ <OPINION> กําไร <PO...,True,True,False,False,False
4,สรุปกระดานรายใหญ่เช้านี้ BGRIM มูลค่าสูงสุด 9 ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> มูลค่า <OPINION> สูงสุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สรุ...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> สูง...,<SYMBOL> BGRIM <ASPECT> ราคาหุ้น <OPINION> มูล...,True,False,False,True,False


In [ ]:
# บันทึกผลลัพธ์เป็นไฟล์ CSV
df.to_csv('/content/fuzzy_wz90.csv', index=False)
# /content/md_fillnote_testing.csv

# **END 💖💚****